In [2]:
import json
import requests
import mysql.connector
from mysql.connector import Error
import pandas as pd

In [3]:
stocks = ['GOOGL', 'META', 'AAPL', 'AMZN']
hourly_data = {}
daily_data = {}
weekly_data = {}

product_data = {
    'url': "https://www.alphavantage.co/query?",
    'functions': ['TIME_SERIES_INTRADAY', 'TIME_SERIES_DAILY'],
    'intraday_interval': '15min',
    'output_size': 'compact',
    'products': ['Shopify', 'IBM'],
    'symbols': ['SHOP.TRT', 'IBM'],
    'APIKey': '7NVAEAZYAXPYKMZB'
}
print(product_data['url'])

#READ Data on Daily Time Series for Shopify - Symbol: SHOP.TRT - API-KEY: 7NVAEAZYAXPYKMZB
TSIurl = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=GOOGL&interval=30min&outputsize=full&apikey=7NVAEAZYAXPYKMZB'
response = requests.get(TSIurl)
timeseries_intraday = response.json()
StockURL = 'https://www.alphavantage.co/query?function=OVERVIEW&symbol=GOOGL&apikey=7NVAEAZYAXPYKMZB'
stocksdata = requests.get(StockURL).json()
#print(timeseries_intraday['Time Series (30min)'].keys())
#print(stocksdata['Name'])

In [4]:
#Create Database Connection
def server_connection(host_name, username, pwd):
    connection = None
    try:
        connection = mysql.connector.connect(host = host_name, user = username, passwd = pwd)
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

server = server_connection('localhost', 'root', '#Snroshan1998')

MySQL Database connection successful


In [5]:
#Create a new database
def create_database(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Database created successfully")
    except Error as err:
        print(f"Error: '{err}'")
        
create_database(server, "CREATE DATABASE SOEN6441")

Error: '1007 (HY000): Can't create database 'soen6441'; database exists'


In [6]:
def create_db_connection(host_name, username, pwd, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(host = host_name, user = username, passwd = pwd, database = db_name)
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

connection = create_db_connection('localhost', 'root', '#Snroshan1998', "SOEN6441")


MySQL Database connection successful


In [7]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")

create_table_stocks = """CREATE TABLE STOCKS (  STOCKNAME VARCHAR(1000),
                                                SYMBOL VARCHAR(20) PRIMARY KEY,
                                                SECTOR VARCHAR(1000),
                                                INDUSTRY VARCHAR(2000),
                                                MARKETCAP BIGINT,
                                                ASSETTYPE VARCHAR(300),
                                                COMPANY_DESCRIPTION VARCHAR(10000)
                                                )"""

execute_query(connection, create_table_stocks)

#Execute a given Query
    
stockinfo = """INSERT INTO STOCKS VALUES ('%s', '%s', '%s', '%s', '%d', '%s', "%s")""" % (stocksdata['Name'], stocksdata['Symbol'], stocksdata['Sector'], stocksdata['Industry'], int(stocksdata['MarketCapitalization']), stocksdata['AssetType'], stocksdata['Description'])
#print(stockinfo)

execute_query(connection, stockinfo)


def add_stocks(stocks):
    for stock in stocks:
        url = 'https://www.alphavantage.co/query?function=OVERVIEW&symbol=%s&apikey=7NVAEAZYAXPYKMZB' % (stock)
        stock_data = requests.get(url).json()
        stock_query= """INSERT INTO STOCKS VALUES ('%s', '%s', '%s', '%s', '%d', '%s', "%s")""" % (stock_data['Name'], stock_data['Symbol'], stock_data['Sector'], stock_data['Industry'], int(stock_data['MarketCapitalization']), stock_data['AssetType'], stock_data['Description'])
        
        execute_query(connection, stock_query)

add_stocks(stocks)


#ID INT AUTO_INCREMENT PRIMARY KEY, 
def create_stock_tables(stocks):
    for stock in stocks:
        create_table = """CREATE TABLE %s ( ID INT PRIMARY KEY,
                                            STOCKSYMBOL VARCHAR(20),
                                            DATENTIME DATETIME NOT NULL,
                                            OPEN FLOAT(9, 4),
                                            HIGH FLOAT(9, 4),
                                            LOW FLOAT(9, 4),
                                            CLOSE FLOAT(9, 4)
                                            )""" % (stock)

        execute_query(connection, create_table)

        
create_stock_tables(stocks)

In [8]:
def get_values(stock, interval, data):
    intraday_values = []
    for datetime in data:
        intraday_values.append([datetime, float(data[datetime]['1. open']), float(data[datetime]['2. high']), float(data[datetime]['3. low']), float(data[datetime]['4. close'])])
    return (intraday_values)

def update_intraday_values(stocks, interval = '15min'):
    for stock in stocks:
        #execute_query(connection, """DELETE FROM %s""" %(stock))
        url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&interval=%s&symbol=%s&outputsize=full&apikey=7NVAEAZYAXPYKMZB' %(interval, stock)
        intraday_data = requests.get(url).json()
        values = get_values(stock, interval, intraday_data['Time Series (%s)' %(interval)])
        closing_data = get_hourly_data(stock, interval, intraday_data['Time Series (%s)' %(interval)])
            #intraday_update_query= """INSERT INTO %s VALUES (%s, '%s', '%s', %f, %f, %f, %f)""" % (stock, n, intraday_data['Meta Data']['2. Symbol'], i[0], i[1], i[2], i[3], i[4])
        #print (closing_data)
        #for n, i in zip(range(1, len(values) + 1), values):
            
            #execute_query(connection, intraday_update_query)
            #execute_query(connection, intraday_update_query)
        #supply values into the query update above and pass into function to execute query
        
                                                                                           
update_intraday_values(stocks)

#UPDATE INTRADAY VALUES FOR ALL STOCKS
def update_intraday_values(stocks, interval = '15min'):
    for stock in stocks:
        execute_query(connection, """DELETE FROM %s""" %(stock))
        url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&interval=%s&symbol=%s&outputsize=full&apikey=7NVAEAZYAXPYKMZB' %(interval, stock)
        intraday_data = requests.get(url).json()
        values = get_values(stock, interval, intraday_data['Time Series (%s)' %(interval)])
        for n, i in zip(range(1, len(values) + 1), values):
            intraday_update_query= """INSERT INTO %s VALUES (%s, '%s', '%s', %f, %f, %f, %f)""" % (stock, n, intraday_data['Meta Data']['2. Symbol'], i[0], i[1], i[2], i[3], i[4])
            execute_query(connection, intraday_update_query)        
                                                                                           
update_intraday_values(stocks)

In [9]:
def read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as err:
        print(f"Error: '{err}'")


#OBTAIN HOURLY DATA
def update_hourly_data(connection, stocks, interval = '15min'):
    for stock in stocks:
        cursor = connection.cursor()
        hourly_data = {}
        
        get_hour = """SELECT DISTINCT DATENTIME FROM {0} WHERE DATENTIME LIKE "%:00:00" """.format(stock)
        get_interval = """SELECT DISTINCT DATENTIME FROM {0} WHERE DATENTIME LIKE "%:00" """.format(stock)
        cursor.execute(get_interval)
        result = cursor.fetchall()
        for i in range(0, 4):
            print(result[i][0])
        #print(str(result[0][0]).split()[0])

        for i in range(len(result)-1):
            open_value = read_query(connection, """SELECT CLOSE FROM {0} WHERE DATENTIME LIKE "{1}" """.format(stock, result[i+1][0]))[0][0]
            close_value = read_query(connection, """SELECT CLOSE FROM {0} WHERE DATENTIME LIKE "{1}" """.format(stock, result[i][0]))[0][0]
            high_value = read_query(connection, """SELECT MAX(HIGH) FROM {0} WHERE DATENTIME LIKE "{1}%" """.format(stock, str(result[0][0]).split()[0]))[0][0]
            low_value = read_query(connection, """SELECT MIN(LOW) FROM {0} WHERE DATENTIME LIKE "{1}%" """.format(stock, str(result[0][0]).split()[0]))[0][0]
            
            hourly_data[result[i][0]] = {"Open": open_value, "High": high_value, "Low": low_value, "Close": close_value}
    
    
    
    
        #print(stock, hourly_data)
       
        
update_hourly_data(connection, ['GOOGL', 'META', 'AAPL', 'AMZN'])

In [10]:
#OBTAIN HOURLY DATA
def update_hourly_data(connection, stocks):
    for stock in stocks:
        cursor = connection.cursor()
        hourly_stock_data = {}
        
        get_hour = """SELECT DISTINCT DATENTIME FROM {0} WHERE DATENTIME LIKE "%:00:00" """.format(stock)
        get_interval = """SELECT DISTINCT DATENTIME FROM {0} WHERE DATENTIME LIKE "%:00" """.format(stock)
       
        cursor.execute(get_hour)
        result = cursor.fetchall()
        cursor.execute(get_interval)
        res1 = cursor.fetchall()
        
        #print(str(result[0][0]).split()[0])

        for i, j in zip(range(len(result)-1), range(4, len(res1)-1, 4)):
            high_value, low_value = [], []
            open_value = read_query(connection, """SELECT CLOSE FROM {0} WHERE DATENTIME LIKE "{1}" """.format(stock, result[i+1][0]))[0][0]
            close_value = read_query(connection, """SELECT CLOSE FROM {0} WHERE DATENTIME LIKE "{1}" """.format(stock, result[i][0]))[0][0]
            for hours in range(j-4, j):
#                intervals.append(res1[hours][0])
                high_value.append(read_query(connection, """SELECT HIGH FROM {0} WHERE DATENTIME LIKE "{1}" """.format(stock, res1[hours][0]))[0][0])
                low_value.append(read_query(connection, """SELECT LOW FROM {0} WHERE DATENTIME LIKE "{1}" """.format(stock, res1[hours][0]))[0][0])
            #print(high_value, low_value)
            hourly_stock_data[result[i][0]] = {"Open": open_value, "High": max(high_value), "Low": min(low_value), "Close": close_value}

        hourly_data[stock] = hourly_stock_data
    print(hourly_data)
    
        
update_hourly_data(connection, ['GOOGL', 'META', 'AAPL', 'AMZN'])

{'GOOGL': {datetime.datetime(2022, 11, 4, 20, 0): {'Open': 86.52, 'High': 86.53, 'Low': 86.45, 'Close': 86.53}, datetime.datetime(2022, 11, 4, 19, 0): {'Open': 86.53, 'High': 86.52, 'Low': 86.45, 'Close': 86.52}, datetime.datetime(2022, 11, 4, 18, 0): {'Open': 86.42, 'High': 86.53, 'Low': 86.3899, 'Close': 86.53}, datetime.datetime(2022, 11, 4, 17, 0): {'Open': 86.64, 'High': 86.75, 'Low': 84.0, 'Close': 86.42}, datetime.datetime(2022, 11, 4, 16, 0): {'Open': 86.25, 'High': 86.64, 'Low': 86.05, 'Close': 86.64}, datetime.datetime(2022, 11, 4, 15, 0): {'Open': 85.36, 'High': 86.265, 'Low': 85.0, 'Close': 86.25}, datetime.datetime(2022, 11, 4, 14, 0): {'Open': 85.16, 'High': 85.48, 'Low': 84.72, 'Close': 85.36}, datetime.datetime(2022, 11, 4, 13, 0): {'Open': 84.405, 'High': 85.5393, 'Low': 84.11, 'Close': 85.16}, datetime.datetime(2022, 11, 4, 12, 0): {'Open': 85.51, 'High': 85.7, 'Low': 84.22, 'Close': 84.405}, datetime.datetime(2022, 11, 4, 11, 0): {'Open': 84.825, 'High': 86.35, 'Low'

for datetime in hourly_data[stock]:
                print(str(datetime).split()[0])
                if(result[i][0] == str(datetime).split()[0]):
                    high_value = [val for val in ]
            print(high_value)
            
            
            
            high = [hourly_data[stock][val]['High'] for val in hourly_data[stock] if(result[i][0] == str(val).split()[0])]
            low = [hourly_data[stock][val]['Low'] for val in hourly_data[stock] if(result[i][0] == str(val).split()[0])]


In [11]:
#OBTAIN DAILY DATA
def update_daily_data(connection, stocks):
    for stock in stocks:
        cursor = connection.cursor()
        daily_stock_data = {}
        
        get_dates = """SELECT DISTINCT DATE(DATENTIME) FROM {0} WHERE DATENTIME LIKE "%20:00:00" """.format(stock)
        cursor.execute(get_dates)
        result = cursor.fetchall()
        
        for i in range(len(result)-1):
            open_value = read_query(connection, """SELECT CLOSE FROM {0} WHERE DATENTIME LIKE "{1}_20:00:00" """.format(stock, result[i+1][0]))[0][0]
            close_value = read_query(connection, """SELECT CLOSE FROM {0} WHERE DATENTIME LIKE "{1}_20:00:00" """.format(stock, result[i][0]))[0][0]
            high = [hourly_data[stock][val]['High'] for val in hourly_data[stock] if(str(result[i][0]) == str(val).split()[0])]
            low = [hourly_data[stock][val]['Low'] for val in hourly_data[stock] if(str(result[i][0]) == str(val).split()[0])]

            daily_stock_data[result[i][0]] = {"Open": open_value, "High": max(high), "Low": min(low), "Close": close_value}

        daily_data[stock] = daily_stock_data
    return daily_data
    
update_daily_data(connection, stocks)

{'GOOGL': {datetime.date(2022, 11, 4): {'Open': 83.68,
   'High': 86.75,
   'Low': 83.222,
   'Close': 86.53},
  datetime.date(2022, 11, 3): {'Open': 87.04,
   'High': 87.97,
   'Low': 83.34,
   'Close': 83.68},
  datetime.date(2022, 11, 2): {'Open': 90.58,
   'High': 91.35,
   'Low': 86.74,
   'Close': 87.04},
  datetime.date(2022, 11, 1): {'Open': 94.8,
   'High': 96.03,
   'Low': 90.26,
   'Close': 90.58},
  datetime.date(2022, 10, 31): {'Open': 96.33,
   'High': 97.03,
   'Low': 93.98,
   'Close': 94.8},
  datetime.date(2022, 10, 28): {'Open': 92.0,
   'High': 96.55,
   'Low': 91.64,
   'Close': 96.33},
  datetime.date(2022, 10, 27): {'Open': 95.35,
   'High': 95.73,
   'Low': 90.38,
   'Close': 92.0},
  datetime.date(2022, 10, 26): {'Open': 97.6001,
   'High': 98.7,
   'Low': 94.35,
   'Close': 95.35},
  datetime.date(2022, 10, 25): {'Open': 102.9,
   'High': 104.82,
   'Low': 96.71,
   'Close': 97.6001},
  datetime.date(2022, 10, 24): {'Open': 101.22,
   'High': 102.97,
   'Low':

#OBTAIN WEEKLY DATA
def update_weekly_data(connection, stocks):
    weekly_stock_data = {}
    for stock in stocks:
        for i, date in zip(range(1, len(daily_data[stock]), 6), daily_data[stock]):
            high_value, low_value = [], []

            for j in range(0, 6):               
                high_value.append(daily_data[stock][date]['High'])
                low_value.append(daily_data[stock][date]['Low'])
                close_value = daily_data[stock][date]['Close']
                open_value = daily_data[stock][date]['Open']
 
            weekly_stock_data[date] = {"Open": open_value, "High": max(high_value), "Low": min(low_value, "Close": close_value}
        weekly_data[stock] = weekly_stock_data
    print(weekly_data)
update_weekly_data(connection, stocks)

In [54]:
#OBTAIN WEEKLY DATA
def update_weekly_data(stocks):
    weekly_stock_data = {}
    for stock in stocks:
        temp = list(daily_data[stock]) #Stores all dates

        for i in range(0, len(temp)-4, 5):
            open_value = daily_data[stock][temp[i+4]]['Open']
            close_value = daily_data[stock][temp[i]]['Close']
            high_value = [daily_data[stock][temp[j]]['High'] for j in range(i, i+5)]
            low_value = [daily_data[stock][temp[j]]['Low'] for j in range(i, i+5)]
            
            weekly_stock_data[temp[i]] = {'Open': open_value, "High": max(high_value), "Low": min(low_value), "Close": close_value}
            weekly_data[stock] = weekly_stock_data
        
        return weekly_data                                                                                            
update_weekly_data(stocks)

{'GOOGL': {datetime.date(2022, 11, 4): {'Open': 96.33,
   'High': 97.03,
   'Low': 83.222,
   'Close': 86.53},
  datetime.date(2022, 10, 28): {'Open': 101.22,
   'High': 104.82,
   'Low': 90.38,
   'Close': 96.33},
  datetime.date(2022, 10, 21): {'Open': 96.3,
   'High': 103.47,
   'Low': 96.319,
   'Close': 101.22},
  datetime.date(2022, 10, 14): {'Open': 98.72,
   'High': 101.14,
   'Low': 94.0,
   'Close': 96.3},
  datetime.date(2022, 10, 7): {'Open': 95.81,
   'High': 102.9254,
   'Low': 95.1615,
   'Close': 98.72}}}

In [34]:
import pandas as pd
print(len(hourly_data)['GOOGL', pd.DataFrame(hourly_data)['GOOGL'][409])    

469 nan
